<a href="https://colab.research.google.com/github/ManuSinghYadav/Mistral-7B-ft-Jawaharlal-Nehru/blob/main/Creating_dataset_from_book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####Here, we will perform basic text preprocessing to convert the book into a dataset of passages.

###Importing libraries and dataset

Downloaded the book "Discovery of India" by Jawaharlal Nehru from [here](https://library.bjp.org/jspui/bitstream/123456789/277/1/The-Discovery-Of-India-Jawaharlal-Nehru.pdf).

Now upload the book here in colab.

In [10]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.3 [186 kB]
Fetched 186 kB in 1s (337 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121753 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.3_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.3) ...
Setting up poppler-utils (22.02.0-2ubuntu0.3) ...
Processing triggers for man-db (2.10.2-1) ...


In [11]:
import pandas as pd

###Convert the book (pdf) into text format.

In [1]:
!mv DOI.pdf doi.pdf

In [ ]:
!pdftotext -nopgbrk doi.pdf

In [4]:
!sed -r ':a /[a-zA-Z,\ ]$/N;s/(.)\n/\1 /;ta' doi.txt | sed 's/.*/\L&/; s/[[:lower:]]\{1\}/\U&/1' | sed 's/\b\([[:alpha:]]\)[[:space:]]\+\([[:alpha:]]\)/\1\2/g' > doi_reformat.txt

In [5]:
#To remove numbers
!sed -i 's/[[:digit:]]\+$//' doi_reformat.txt

Now download the file and remo:ve the index portion manually and again reupload it.

###Gather chapters to target

In [12]:
with open("doi_reformat.txt") as f:
    file_content = f.read()

chapters=file_content.split("\n\n")

In [13]:
chapters[0]

"It is more than twenty months since we we re brought here, more than twenty months of my ninth term of imprisonment.\nTh enew moon, ashimmering crescent in the darkening sky, greeted us on our arrival here. th ebright fortnight of the waxing moon ha dbegun. ever since then each coming of the new moon has been areminder to me that another mo nt hof my imprisonment is over. so it was with my last term of imprisonment which began with the new moon, just after the deepavali, the festival of light. th emoon, ever acompanion to me in prison, has grown more friendly with closer acquaintance, areminder of the loveliness of this world, of the waxing and waning of life, of light following darkness, of death and resurrection following each other in interminable succession. ever changing, yet ever the same, ihave watched it in its different phases and its many moods in the evening, as the shadows lengthen, in the still hours of the night, an dwhen the breath an dwhisper of dawn bring promise of t

In [14]:
len(chapters)

680

###Sepreating each paragraphs

In [15]:
# Define minimum and maximum lengths to ensure a good interview flow
passage_minlen=300  # if paragraph <300 chars -> merge with next
passage_maxlen=2000  # if paragraph >2k chars -> split

In [16]:
# Process the chapters into suitable interview passages
passages=[]
for chap in chapters:
    passage=""
    for par in chap.split("\n"):
        if(len(passage)<passage_minlen) or not passage[-1]=="." and len(passage)<passage_maxlen:
            passage+="\n" + par
        else:
            passages.append(passage.strip().replace("\n", " "))
            passage=par


In [17]:
passages[409]

'The present and found joy and harmony in the beauty they saw around them or which they themselves created. the indians found this joy and harmony also in the present, but, at the same time, their eyes were turned towards deeper knowledge and their minds trafficked with strange questions. the chinese, fully aware of these questions and their mystery, in their wisdom avoided entanglement with them. in their different ways each tried to express the fullness and beauty of life. history has shown that india and china had stronger foundations and greater staying power; they have thus far survived, though they have been badly shaken and have greatly deteriorated, and the future is obscure.'

In [18]:
len(passages)

1671

###Creating a dataframe of the passages

In [21]:
df = pd.DataFrame({"Passages": passages})

In [22]:
df

,Passages
0,It is more than twenty months since we we re b...
1,It is an easy calendar (though it must be adju...
2,"Th en newspapers were allowed and, some weeks ..."
3,Th er ewere ma ny thousands of prisoners of wa...
4,"Famine famine came, ghastly, staggering, horri..."
...,...
1666,Was far more than we had imagined. these three...
1667,The smooth surface was ruffled and cracks appe...
1668,The war ended and the atom bomb became the sym...
1669,"Akbar, emperor, 35, 52, 142, 242, 257-61, 263-..."


Export this dataframe to CSV and download it so that we'll have it handy for use in upcoming notebooks.

In [23]:
df.to_csv("doi_passages.csv", index=False)